In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
from pathlib import Path
import pywt
from tqdm import tqdm

In [2]:
import numpy as np
from scipy.ndimage import label, binary_closing, generate_binary_structure

def detect_bouts(denoised_signal, window_size=5, closing_iterations=1, 
                 min_bout_length=10, threshold_multiplier=3):
    """
    Detect increasing/decreasing bouts in a denoised signal.
    
    Parameters:
        denoised_signal (array): Denoised 1D signal
        window_size (int): Size of smoothing window for gradient
        closing_iterations (int): Number of iterations for binary closing
        min_bout_length (int): Minimum bout duration in samples
        threshold_multiplier (float): MAD multiplier for adaptive threshold
        
    Returns:
        list: List of bout dictionaries with start/end indices and type
    """
    # Compute gradient
    slope = np.gradient(denoised_signal)
    
    # Smooth gradient
    if window_size > 1:
        kernel = np.ones(window_size)/window_size
        smoothed_slope = np.convolve(slope, kernel, mode='same')
    else:
        smoothed_slope = slope
    
    # Calculate adaptive threshold using MAD
    mad = np.median(np.abs(smoothed_slope - np.median(smoothed_slope)))
    slope_threshold = threshold_multiplier * mad
    
    # Thresholding
    increasing = smoothed_slope > slope_threshold
    decreasing = smoothed_slope < -slope_threshold
    
    # Morphological closing to merge nearby regions
    struct = generate_binary_structure(1, 1)
    increasing = binary_closing(increasing, structure=struct, iterations=closing_iterations)
    decreasing = binary_closing(decreasing, structure=struct, iterations=closing_iterations)
    
    # Label regions
    inc_labels, n_inc = label(increasing)
    dec_labels, n_dec = label(decreasing)
    
    # Extract bouts from labels
    def _get_bouts(labels, n_labels):
        bouts = []
        for i in range(1, n_labels+1):
            indices = np.where(labels == i)[0]
            if len(indices) == 0:
                continue
            start = indices[0]
            end = indices[-1]
            if (end - start + 1) >= min_bout_length:
                bouts.append((start, end))
        return bouts
    
    inc_bouts = _get_bouts(inc_labels, n_inc)
    dec_bouts = _get_bouts(dec_labels, n_dec)
    
    # Combine and format results
    all_bouts = []
    for start, end in inc_bouts:
        all_bouts.append([start, end])
    for start, end in dec_bouts:
        all_bouts.append([start, end])
    
    # Sort by start index
    all_bouts.sort(key=lambda x: x[0])
    
    return all_bouts

In [3]:
def cropping(video_path, output_folder, episodes):
    import cv2 
    if Path(f'{output_folder}').exists():
        import shutil
        shutil.rmtree(f'{output_folder}')
    
    Path(output_folder).mkdir(parents=True, exist_ok = True)
    fps = 200
    frame_adjustment = int(fps * 1)  # 1 second worth of frames
    
    # Open the video
    cap = cv2.VideoCapture(video_path)
    
    # Get video properties
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Function to add text to a frame
    def add_text_to_frame(frame, text):
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        font_color = (255, 255, 255)  # White color
        font_thickness = 2
        cv2.putText(frame, text, (50, 50), font, font_scale, font_color, font_thickness)
        return frame
    
    # Iterate through the frame ranges
    for i, (start_frame, end_frame) in enumerate(episodes):
        # Adjust start and end frames
        adjusted_start = max(0, start_frame - frame_adjustment)
        adjusted_end = min(total_frames - 1, end_frame + frame_adjustment)
    
        # Set the start frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, adjusted_start)
    
        # Define the output video writer
        output_path = f'{output_folder}clip_{i}_{int(start_frame/fps)}-{int(end_frame/fps)}.mp4'
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps * 0.5, (width, height))
    
        # Loop through the frames in the range
        for frame_num in range(adjusted_start, adjusted_end + 1):
            ret, frame = cap.read()
            if not ret:
                break
    
            # Add "episode" text to the frame
            if frame_num in list(range(start_frame, end_frame)):
                frame = add_text_to_frame(frame, "episode")
    
            # Write the frame to the output video
            out.write(frame)
    
        # Release the output video writer
        out.release()
        print(f"clips {i} is ready")
    
    # Release the video capture object
    cap.release()
    
    print("Clips have been successfully created.")

In [4]:
def wavelet_denoise(signal, wavelet='bior3.1', level=3, threshold_type='soft'): # pywt.wavelist(kind='discrete')
    coeffs = pywt.wavedec(signal, wavelet, mode='per', level=level)  # Decompose
    sigma = np.median(np.abs(coeffs[-level])) / 0.6745  # Noise estimation (MAD)
    threshold = sigma * np.sqrt(2 * np.log(len(signal)))  # Universal threshold
    coeffs_thresh = [pywt.threshold(c, threshold, mode=threshold_type) for c in coeffs]
    return pywt.waverec(coeffs_thresh, wavelet, mode='per')  # Reconstruct

In [5]:
folders = glob("F:\\swim_motion_data\\lcr_kai\\*")

for folder in tqdm(folders[18:]):

    profix = folder.split("\\")[-1]
    print(profix)
    save_dir = f'F:\\swim_motion_data\\lcr_kai_v4\\{profix}'
    Path(save_dir).mkdir(parents=True, exist_ok=True)
    
    date = profix.split("_")[0]
    fish_no = profix.split("_")[1]
    set_no = profix.split("-")[2][3:]
    trial_no = profix.split("-")[-1][1:]
    
    h5_path = f"F:\\Temp_opto\\{date}\\lcr\\TOPCAMERA\\{fish_no}\\{set_no}\\{date}_{set_no}_Trial{trial_no}.h5"
    #### load top cam data
    top_df = pd.read_csv(f"{folder}\\TOPCAM\\y_shift_idx.csv")
    y_motions = np.load(f"{folder}\\TOPCAM\\y_shift.npy")
    up_idxs = top_df["ori"].values[top_df["is_up"].values == 1]
    down_idxs = top_df["ori"].values[top_df["is_up"].values == -1]
    
    #### load side cam data
    side_df = pd.read_csv(f"{folder}\\SIDECAM\\x_shift_idx.csv")
    x_motions = np.load(f"{folder}\\SIDECAM\\x_shift.npy")
    left_idxs = side_df["ori"].values[side_df["is_left"].values == 1]
    right_idxs = side_df["ori"].values[side_df["is_left"].values == -1]
    
    #### load old episode
    top_episodes = np.load(f"{folder}\\TOPCAM\\episodes_idx.npy")
    side_episodes = np.load(f"{folder}\\SIDECAM\\episodes_idx.npy")
    
    filt_y = wavelet_denoise(y_motions)
    filt_x = wavelet_denoise(x_motions)
    new_top_episodes = detect_bouts(filt_y,
                                    window_size=5, # Controls gradient smoothing
                                    closing_iterations=4, # Controls gap merging
                                    min_bout_length=10, # Filters short fluctuations
                                    threshold_multiplier=3) # Adjusts sensitivity
    new_side_episodes = detect_bouts(filt_x,
                                    window_size=5, # Controls gradient smoothing
                                    closing_iterations=4, # Controls gap merging
                                    min_bout_length=10, # Filters short fluctuations
                                    threshold_multiplier=3) # Adjusts sensitivity

    cropping(h5_path.split('.')[0]+".mp4",f"{save_dir}\\TOPCAM\\clips\\", new_top_episodes)
    sidecam_path = f"{h5_path.split("\\TO")[0]}\\SIDECAMERA\\{fish_no}\\{set_no}\\FLIR_{profix}-0000.avi"
    cropping(sidecam_path, f"{save_dir}\\SIDECAM\\clips\\", new_side_episodes)
    np.save(f"{save_dir}\\TOPCAM\\events.npy" , new_top_episodes)
    np.save(f"{save_dir}\\SIDECAM\\events.npy" ,new_top_episodes)

  0%|                                                                                           | 0/25 [00:00<?, ?it/s]

2024-08-07_F1_01-T2
clips 0 is ready
clips 1 is ready
clips 2 is ready
clips 3 is ready
clips 4 is ready
clips 5 is ready
clips 6 is ready
clips 7 is ready
clips 8 is ready
clips 9 is ready
clips 10 is ready
clips 11 is ready
clips 12 is ready
clips 13 is ready
clips 14 is ready
clips 15 is ready
clips 16 is ready
clips 17 is ready
clips 18 is ready
clips 19 is ready
clips 20 is ready
clips 21 is ready
clips 22 is ready
clips 23 is ready
clips 24 is ready
clips 25 is ready
clips 26 is ready
clips 27 is ready
clips 28 is ready
clips 29 is ready
clips 30 is ready
clips 31 is ready
clips 32 is ready
clips 33 is ready
clips 34 is ready
clips 35 is ready
clips 36 is ready
clips 37 is ready
clips 38 is ready
clips 39 is ready
clips 40 is ready
clips 41 is ready
clips 42 is ready
clips 43 is ready
clips 44 is ready
clips 45 is ready
clips 46 is ready
clips 47 is ready
clips 48 is ready
clips 49 is ready
clips 50 is ready
clips 51 is ready
clips 52 is ready
clips 53 is ready
clips 54 is ready


  4%|███▏                                                                            | 1/25 [08:14<3:17:40, 494.19s/it]

clips 277 is ready
Clips have been successfully created.
2024-08-07_F1_omr-T1
clips 0 is ready
clips 1 is ready
clips 2 is ready
clips 3 is ready
clips 4 is ready
clips 5 is ready
clips 6 is ready
clips 7 is ready
clips 8 is ready
clips 9 is ready
clips 10 is ready
clips 11 is ready
clips 12 is ready
clips 13 is ready
clips 14 is ready
clips 15 is ready
clips 16 is ready
clips 17 is ready
clips 18 is ready
clips 19 is ready
clips 20 is ready
clips 21 is ready
clips 22 is ready
clips 23 is ready
clips 24 is ready
clips 25 is ready
clips 26 is ready
clips 27 is ready
clips 28 is ready
clips 29 is ready
clips 30 is ready
clips 31 is ready
clips 32 is ready
clips 33 is ready
clips 34 is ready
clips 35 is ready
clips 36 is ready
clips 37 is ready
clips 38 is ready
clips 39 is ready
clips 40 is ready
clips 41 is ready
clips 42 is ready
clips 43 is ready
clips 44 is ready
clips 45 is ready
clips 46 is ready
clips 47 is ready
clips 48 is ready
clips 49 is ready
clips 50 is ready
clips 51 is re

  8%|██████▍                                                                         | 2/25 [09:29<1:35:04, 248.02s/it]

...

clips 22 is ready
Clips have been successfully created.
2024-08-29_F4_01-T8
clips 0 is ready
clips 1 is ready
clips 2 is ready
clips 3 is ready
clips 4 is ready
clips 5 is ready
clips 6 is ready
clips 7 is ready
clips 8 is ready
clips 9 is ready
clips 10 is ready
clips 11 is ready
clips 12 is ready
clips 13 is ready
clips 14 is ready
clips 15 is ready
clips 16 is ready
clips 17 is ready
clips 18 is ready
clips 19 is ready
clips 20 is ready
clips 21 is ready
clips 22 is ready
clips 23 is ready
clips 24 is ready
clips 25 is ready
clips 26 is ready
clips 27 is ready
clips 28 is ready
clips 29 is ready
clips 30 is ready
clips 31 is ready
clips 32 is ready
clips 33 is ready
clips 34 is ready
clips 35 is ready
clips 36 is ready
clips 37 is ready
Clips have been successfully created.
clips 0 is ready
clips 1 is ready
clips 2 is ready
clips 3 is ready
clips 4 is ready
clips 5 is ready
clips 6 is ready
clips 7 is ready
clips 8 is ready
clips 9 is ready
clips 10 is ready
clips 11 is ready
clips 

 96%|████████████████████████████████████████████████████████████████████████████▊   | 24/25 [1:17:54<00:33, 33.81s/it]

clips 18 is ready
Clips have been successfully created.
2024-08-29_F4_01-T9
clips 0 is ready
clips 1 is ready
clips 2 is ready
clips 3 is ready
clips 4 is ready
clips 5 is ready
clips 6 is ready
clips 7 is ready
clips 8 is ready
clips 9 is ready
clips 10 is ready
clips 11 is ready
clips 12 is ready
clips 13 is ready
clips 14 is ready
clips 15 is ready
clips 16 is ready
clips 17 is ready
clips 18 is ready
clips 19 is ready
clips 20 is ready
clips 21 is ready
clips 22 is ready
clips 23 is ready
clips 24 is ready
clips 25 is ready
clips 26 is ready
clips 27 is ready
clips 28 is ready
clips 29 is ready
clips 30 is ready
clips 31 is ready
clips 32 is ready
clips 33 is ready
clips 34 is ready
clips 35 is ready
clips 36 is ready
clips 37 is ready
clips 38 is ready
clips 39 is ready
clips 40 is ready
clips 41 is ready
clips 42 is ready
clips 43 is ready
clips 44 is ready
clips 45 is ready
clips 46 is ready
clips 47 is ready
Clips have been successfully created.
clips 0 is ready
clips 1 is read

100%|███████████████████████████████████████████████████████████████████████████████| 25/25 [1:18:35<00:00, 188.61s/it]

clips 36 is ready
Clips have been successfully created.
